In [1]:
import time
import requests
import json

import os
from getpass import getpass

# Get your API key from https://cloud.ionq.com/settings/keys
api_key = os.getenv('IONQ_API_KEY') or getpass('Enter your IonQ API key: ')

In [2]:
def submit_job(headers, data):
    url = "https://api.ionq.co/v0.3/jobs"
    response = requests.post(url, headers=headers, data=data)
    response_json = response.json()
    assert response.status_code == 200, f"Error: {response_json.get('message', 'Unknown error')}"
    return response_json["id"]

def query_job(job_id, headers):
    url = f"https://api.ionq.co/v0.3/jobs/{job_id}"
    response = requests.get(url, headers=headers)
    response_json = response.json()
    assert response.status_code == 200, f"Error: {response_json.get('message', 'Unknown error')}"
    return response_json["status"]

def get_job_results(job_id, headers):
    url = f"https://api.ionq.co/v0.3/jobs/{job_id}/results"
    response = requests.get(url, headers=headers)
    response_json = response.json()
    assert response.status_code == 200, f"Error: {response_json.get('message', 'Unknown error')}"
    return response_json

In [3]:
headers = {
    "Authorization": f"apiKey {api_key}",
    "Content-Type": "application/json",
}

data = open("data.json").read()

# print(json.dumps(data, indent=4))

```json
{
    "shots": 100,
    "target": "simulator",
    "input": {
        "format": "ionq.circuit.v0",
        "gateset": "qis",
        "qubits": 2,
        "circuit": [
            {
                "gate": "h",
                "target": 0
            },
            {
                "gate": "cnot",
                "control": 0,
                "target": 1
            }
        ]
    }
}
```

In [4]:
job_id = submit_job(headers, data)

status = "ready"
while status != "completed":
    time.sleep(1)  # wait for 1 second before querying again
    status = query_job(job_id, headers)

results = get_job_results(job_id, headers)
print(results)

{'0': 0.5, '3': 0.5}
